In [0]:
%sql

-- By Category --

WITH base_cnos AS (
  SELECT
    category,
    quarter,
    month,
    CAST(SUBSTRING(month, 6, 2) AS INT) AS month_num,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END) AS cnos,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(target           AS DOUBLE) END) AS cnos_target,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(ya_actual        AS DOUBLE) END) AS cnos_ya
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month >= '2025M07' AND month <= '2025M12'
    AND quarter IN ('JAS','OND')
  GROUP BY category, quarter, month
),

-- ---------- Per-category rollups (C-NOS) ----------
quarter_rollup AS (
  SELECT
    category,
    quarter,
    SUM(cnos)        AS q_cnos,
    SUM(cnos_target) AS q_target,
    SUM(cnos_ya)     AS q_ya
  FROM base_cnos
  GROUP BY category, quarter
),

half_year AS (
  SELECT
    category,
    SUM(cnos)        AS hy_cnos,
    SUM(cnos_target) AS hy_target,
    SUM(cnos_ya)     AS hy_ya
  FROM base_cnos
  GROUP BY category
),

month_wide AS (
  SELECT
    category,
    MAX(CASE WHEN month = '2025M07' THEN cnos END)/1000/1000/150.1 AS JULY_MM,
    MAX(CASE WHEN month = '2025M07' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS JULY_IYA,
    MAX(CASE WHEN month = '2025M08' THEN cnos END)/1000/1000/150.1 AS AUG_MM,
    MAX(CASE WHEN month = '2025M08' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS AUG_IYA,
    MAX(CASE WHEN month = '2025M09' THEN cnos END)/1000/1000/150.1 AS SEP_MM,
    MAX(CASE WHEN month = '2025M09' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS SEP_IYA,
    MAX(CASE WHEN month = '2025M10' THEN cnos END)/1000/1000/150.1 AS OCT_MM,
    MAX(CASE WHEN month = '2025M10' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS OCT_IYA,
    MAX(CASE WHEN month = '2025M11' THEN cnos END)/1000/1000/150.1 AS NOV_MM,
    MAX(CASE WHEN month = '2025M11' THEN cnos/NULLIF(cnos_ya,0)*100 END) AS NOV_IYA,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END)/1000/1000/150.1 AS 1H_to_date,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END) /
    NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos_ya END),0)*100 AS 1H_to_date_IYA
  FROM base_cnos
  GROUP BY category
),

category_metrics AS (
  SELECT
    m.category,
    MAX(CASE WHEN q.quarter = 'JAS' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS JAS_IYA,
    MAX(CASE WHEN q.quarter = 'OND' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS OND_IYA,
    (h.hy_target)/1000/1000/150.1                                             AS 1H_MM,
    (h.hy_target/NULLIF(h.hy_ya,0))*100                                       AS 1H_IYA,
    m.JULY_MM         AS JUL_MM,
    m.JULY_IYA        AS JUL_IYA,
    m.AUG_MM          AS AUG_MM,  
    m.AUG_IYA         AS AUG_IYA,
    m.SEP_MM          AS SEP_MM,  
    m.SEP_IYA         AS SEP_IYA,
    m.OCT_MM          AS OCT_MM,  
    m.OCT_IYA         AS OCT_IYA,
    m.NOV_MM          AS NOV_MM,  
    m.NOV_IYA         AS NOV_IYA,
    m.1H_to_date      AS 1H_To_Date_MM,
    m.1H_to_date_IYA  AS 1H_To_Date_IYA
  FROM month_wide m
  LEFT JOIN quarter_rollup q ON q.category = m.category
  LEFT JOIN half_year      h ON h.category = m.category
  GROUP BY m.category, h.hy_target, h.hy_cnos, h.hy_ya,
           m.JULY_MM, m.JULY_IYA, m.AUG_MM, m.AUG_IYA,
           m.SEP_MM, m.SEP_IYA, m.OCT_MM, m.OCT_IYA,m.NOV_MM,m.NOV_IYA,
           m.1H_to_date, m.1H_to_date_IYA
),

-- ---------- TOTAL rollup ----------
t_quarter AS (
  SELECT
    'TOTAL' AS category,
    quarter,
    SUM(cnos)        AS q_cnos,
    SUM(cnos_target) AS q_target,
    SUM(cnos_ya)     AS q_ya
  FROM base_cnos
  GROUP BY quarter
),

t_half_year AS (
  SELECT
    'TOTAL' AS category,
    SUM(cnos)        AS hy_cnos,
    SUM(cnos_target) AS hy_target,
    SUM(cnos_ya)     AS hy_ya
  FROM base_cnos
),

t_month_wide AS (
  SELECT
    'TOTAL' AS category,
    SUM(CASE WHEN month = '2025M07' THEN cnos END)/1000/1000/150.1 AS JULY_MM,
    SUM(CASE WHEN month = '2025M07' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M07' THEN cnos_ya END),0) * 100 AS JULY_IYA,
    SUM(CASE WHEN month = '2025M08' THEN cnos END)/1000/1000/150.1 AS AUG_MM,
    SUM(CASE WHEN month = '2025M08' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M08' THEN cnos_ya END),0) * 100 AS AUG_IYA,
    SUM(CASE WHEN month = '2025M09' THEN cnos END)/1000/1000/150.1 AS SEP_MM,
    SUM(CASE WHEN month = '2025M09' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M09' THEN cnos_ya END),0) * 100 AS SEP_IYA,
    SUM(CASE WHEN month = '2025M10' THEN cnos END)/1000/1000/150.1 AS OCT_MM,
    SUM(CASE WHEN month = '2025M10' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M10' THEN cnos_ya END),0) * 100 AS OCT_IYA,
    SUM(CASE WHEN month = '2025M11' THEN cnos END)/1000/1000/150.1 AS NOV_MM,
    SUM(CASE WHEN month = '2025M11' THEN cnos END) / NULLIF(SUM(CASE WHEN month = '2025M11' THEN cnos_ya END),0) * 100 AS NOV_IYA,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END)/1000/1000/150.1 AS 1H_to_date,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END) /
    NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos_ya END),0)*100 AS 1H_to_date_IYA
  FROM base_cnos
),

total_metrics AS (
  SELECT
    m.category,
    MAX(CASE WHEN q.quarter = 'JAS' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS JAS_IYA,
    MAX(CASE WHEN q.quarter = 'OND' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS OND_IYA,
    (h.hy_target)/1000/1000/150.1                                             AS 1H_MM,
    (h.hy_target/NULLIF(h.hy_ya,0))*100                                       AS 1H_IYA,
    m.JULY_MM         AS JUL_MM,
    m.JULY_IYA        AS JUL_IYA,
    m.AUG_MM          AS AUG_MM,  
    m.AUG_IYA         AS AUG_IYA,
    m.SEP_MM          AS SEP_MM,  
    m.SEP_IYA         AS SEP_IYA,
    m.OCT_MM          AS OCT_MM,  
    m.OCT_IYA         AS OCT_IYA,
    m.NOV_MM          AS NOV_MM,  
    m.NOV_IYA         AS NOV_IYA,
    m.1H_to_date      AS 1H_To_Date_MM,
    m.1H_to_date_IYA  AS 1H_To_Date_IYA
  FROM t_month_wide m
  LEFT JOIN t_quarter q ON q.category = m.category
  LEFT JOIN t_half_year h ON h.category = m.category
  GROUP BY m.category, h.hy_target, h.hy_cnos, h.hy_ya,
           m.JULY_MM, m.JULY_IYA, m.AUG_MM, m.AUG_IYA,
           m.SEP_MM, m.SEP_IYA, m.OCT_MM, m.OCT_IYA,m.NOV_MM,m.NOV_IYA,
           m.1H_to_date, m.1H_to_date_IYA
),

-- ---------- GIV ----------
giv_category AS (
  SELECT
    category,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv END) /
    NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv_ya END),0) * 100
      AS GIV_1H_to_date_IYA
  FROM (
    SELECT
      category,
      month,
      SUM(CASE WHEN account = 'GIV' THEN CAST(forecast_act_est AS DOUBLE) END) AS giv,
      SUM(CASE WHEN account = 'GIV' THEN CAST(ya_actual        AS DOUBLE) END) AS giv_ya
    FROM hive_metastore.japan_gold.cnos_vw
    WHERE month >= '2025M07' AND month <= '2025M12'
    GROUP BY category, month
  ) s
  GROUP BY category
),

giv_total AS (
  SELECT
    'TOTAL' AS category,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv END) /
    NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv_ya END),0) * 100
      AS GIV_1H_to_date_IYA
  FROM (
    SELECT
      month,
      SUM(CASE WHEN account = 'GIV' THEN CAST(forecast_act_est AS DOUBLE) END) AS giv,
      SUM(CASE WHEN account = 'GIV' THEN CAST(ya_actual        AS DOUBLE) END) AS giv_ya
    FROM hive_metastore.japan_gold.cnos_vw
    WHERE month >= '2025M07' AND month <= '2025M12'
    GROUP BY month
  ) s
),

giv_union AS (
  SELECT * FROM giv_category
  UNION ALL
  SELECT * FROM giv_total
),

final_union AS (
  SELECT
    f.*,
    g.GIV_1H_to_date_IYA
  FROM (
    SELECT * FROM category_metrics
    UNION ALL
    SELECT * FROM total_metrics
  ) f
  LEFT JOIN giv_union g
    ON g.category = f.category
)

-- ---------- Final output (only till POC) ----------
SELECT
  category as `By Category` ,
    ROUND(JAS_IYA, 0) AS JAS_IYA,
    ROUND(OND_IYA, 0) AS OND_IYA,
    ROUND(`1H_MM`, 0) AS `1H_MM`,
    ROUND(`1H_IYA`, 0) AS `1H_IYA`,
    ROUND(JUL_MM, 0) AS JUL_MM,
    ROUND(JUL_IYA, 0) AS JUL_IYA,
    ROUND(AUG_MM, 0) AS AUG_MM,
    ROUND(AUG_IYA, 0) AS AUG_IYA,
    ROUND(SEP_MM, 0) AS SEP_MM,
    ROUND(SEP_IYA, 0) AS SEP_IYA,
    ROUND(OCT_MM, 0) AS OCT_MM,
    ROUND(OCT_IYA, 0) AS OCT_IYA,
    ROUND(NOV_MM, 0) AS NOV_MM,
    ROUND(NOV_IYA, 0) AS NOV_IYA, 
    ROUND(1H_To_Date_MM, 0) AS 1H_To_Date_MM,
    ROUND(1H_To_Date_IYA, 0) AS 1H_To_Date_IYA,
  ROUND(`1H_To_Date_IYA`-`1H_IYA` , 0) AS `vs_Target`,      
  ROUND(GIV_1H_to_date_IYA, 0) AS `GIV_1H_to_date_IYA`,            
  ROUND(`1H_To_Date_IYA` - GIV_1H_to_date_IYA, 0) AS `Gap_VS._C-NOS`  
FROM final_union
WHERE category IN (
    'TOTAL','Detergent','Fabric Enhancer','Dish','Air','Baby','Fem','Hair','Shave','Oral','Braun','POC'
)
ORDER BY
  CASE category
    WHEN 'TOTAL'           THEN 0
    WHEN 'Detergent'       THEN 1
    WHEN 'Fabric Enhancer' THEN 2
    WHEN 'Dish'            THEN 3
    WHEN 'Air'             THEN 4 
    WHEN 'Baby'            THEN 5 
    WHEN 'Fem'             THEN 6
    WHEN 'Hair'            THEN 7
    WHEN 'Shave'           THEN 8
    WHEN 'Oral'            THEN 9
    WHEN 'Braun'           THEN 10
    WHEN 'POC'             THEN 11
    ELSE 99
  END ASC;

In [0]:
%sql

-- By Team --

WITH base_cnos_detail AS (
  SELECT
    CASE
      WHEN org2 = 'Division 2' AND org4 IN ('DP 1','DP 2','DP 3','DP 4','DP 5') THEN '   DP'
      WHEN org2 = 'Division 2' AND org4 IN ('MHC 1','MHC 2','MHC 3','MHC 4','MHC 5') THEN '   MHC'
      WHEN org2 = 'Division 2' AND org4 IN ('HSM 1','HSM 2','HSM 3')              THEN '   HSM'
      WHEN org2 = 'Division 2' AND org4 IN ('WS 1','WS 2','WS 3')                 THEN '   WS'
      WHEN org2 = 'Division 1'                                                  THEN 'Div 1'
      WHEN org2 <> 'Division 2' AND org2 <> 'Division 1'                         THEN org2
      ELSE NULL
    END AS team,
    quarter,
    month,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS cnos,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS cnos_ya
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month BETWEEN '2025M07' AND '2025M12'
    AND quarter IN ('JAS','OND')
  GROUP BY
    CASE
      WHEN org2 = 'Division 2' AND org4 IN ('DP 1','DP 2','DP 3','DP 4','DP 5') THEN '   DP'
      WHEN org2 = 'Division 2' AND org4 IN ('MHC 1','MHC 2','MHC 3','MHC 4','MHC 5') THEN '   MHC'
      WHEN org2 = 'Division 2' AND org4 IN ('HSM 1','HSM 2','HSM 3')              THEN '   HSM'
      WHEN org2 = 'Division 2' AND org4 IN ('WS 1','WS 2','WS 3')                 THEN '   WS'
      WHEN org2 = 'Division 1'                                                  THEN 'Div 1'
      WHEN org2 <> 'Division 2' AND org2 <> 'Division 1'                         THEN org2
      ELSE NULL
    END,
    quarter, month
),

base_cnos_detail_clean AS (
  SELECT * FROM base_cnos_detail WHERE team IS NOT NULL
),

base_cnos_div2 AS (
  SELECT
    'Div 2' AS team,  -- label changed
    quarter,
    month,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS cnos,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account = 'C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS cnos_ya
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month BETWEEN '2025M07' AND '2025M12'
    AND quarter IN ('JAS','OND')
    AND org2 = 'Division 2'  -- keep raw filter
  GROUP BY quarter, month
),

base_cnos AS (
  SELECT * FROM base_cnos_detail_clean
  UNION ALL
  SELECT * FROM base_cnos_div2
),

quarter_rollup AS (
  SELECT
    team, quarter,
    SUM(cnos)        AS q_cnos,
    SUM(cnos_target) AS q_target,
    SUM(cnos_ya)     AS q_ya
  FROM base_cnos
  GROUP BY team, quarter
),

half_year AS (
  SELECT
    team,
    SUM(cnos)        AS hy_cnos,
    SUM(cnos_target) AS hy_target,
    SUM(cnos_ya)     AS hy_ya
  FROM base_cnos
  GROUP BY team
),

/* avoid numeric-leading aliases in intermediate CTEs */
month_wide AS (
  SELECT
    team,
    SUM(CASE WHEN month = '2025M07' THEN cnos END)/1000/1000/150.1 AS JULY_MM,
    SUM(CASE WHEN month = '2025M07' THEN cnos END) /
      NULLIF(SUM(CASE WHEN month = '2025M07' THEN cnos_ya END),0) * 100 AS JULY_IYA,

    SUM(CASE WHEN month = '2025M08' THEN cnos END)/1000/1000/150.1 AS AUG_MM,
    SUM(CASE WHEN month = '2025M08' THEN cnos END) /
      NULLIF(SUM(CASE WHEN month = '2025M08' THEN cnos_ya END),0) * 100 AS AUG_IYA,

    SUM(CASE WHEN month = '2025M09' THEN cnos END)/1000/1000/150.1 AS SEP_MM,
    SUM(CASE WHEN month = '2025M09' THEN cnos END) /
      NULLIF(SUM(CASE WHEN month = '2025M09' THEN cnos_ya END),0) * 100 AS SEP_IYA,

    SUM(CASE WHEN month = '2025M10' THEN cnos END)/1000/1000/150.1 AS OCT_MM,
    SUM(CASE WHEN month = '2025M10' THEN cnos END) /
      NULLIF(SUM(CASE WHEN month = '2025M10' THEN cnos_ya END),0) * 100 AS OCT_IYA,

    SUM(CASE WHEN month = '2025M11' THEN cnos END)/1000/1000/150.1 AS NOV_MM,
    SUM(CASE WHEN month = '2025M11' THEN cnos END) /
      NULLIF(SUM(CASE WHEN month = '2025M11' THEN cnos_ya END),0) * 100 AS NOV_IYA,

    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END)/1000/1000/150.1 AS H1_to_date_MM,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END) /
      NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos_ya END),0) * 100 AS H1_to_date_IYA
  FROM base_cnos
  GROUP BY team
),

team_metrics AS (
  SELECT
    m.team,
    MAX(CASE WHEN q.quarter = 'JAS' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS JAS_IYA,
    MAX(CASE WHEN q.quarter = 'OND' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS OND_IYA,
    h.hy_target/1000/1000/150.1 AS `1H_MM`,
    h.hy_target/NULLIF(h.hy_ya,0)*100 AS `1H_IYA`,
    m.JULY_MM        AS JUL_MM,
    m.JULY_IYA       AS JUL_IYA,
    m.AUG_MM         AS AUG_MM,
    m.AUG_IYA        AS AUG_IYA,
    m.SEP_MM         AS SEP_MM,
    m.SEP_IYA        AS SEP_IYA,
    m.OCT_MM         AS OCT_MM,
    m.OCT_IYA        AS OCT_IYA,
    m.NOV_MM         AS NOV_MM,
    m.NOV_IYA        AS NOV_IYA,
    m.H1_to_date_MM  AS `1H_To_Date_MM`,
    m.H1_to_date_IYA AS `1H_To_Date_IYA`
  FROM month_wide m
  LEFT JOIN quarter_rollup q ON q.team = m.team
  LEFT JOIN half_year      h ON h.team = m.team
  GROUP BY m.team, h.hy_target, h.hy_cnos, h.hy_ya,
           m.JULY_MM, m.JULY_IYA, m.AUG_MM, m.AUG_IYA,
           m.SEP_MM, m.SEP_IYA, m.OCT_MM, m.OCT_IYA, m.NOV_MM, m.NOV_IYA,
           m.H1_to_date_MM, m.H1_to_date_IYA
),

/* TOTAL rows for C‑NOS */
t_quarter AS (
  SELECT
    'TOTAL' AS team,
    quarter,
    SUM(CASE WHEN account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS q_cnos,
    SUM(CASE WHEN account='C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS q_target,
    SUM(CASE WHEN account='C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS q_ya
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month BETWEEN '2025M07' AND '2025M12'
    AND quarter IN ('JAS','OND')
  GROUP BY quarter
),

t_half_year AS (
  SELECT
    'TOTAL' AS team,
    SUM(CASE WHEN account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS hy_cnos,
    SUM(CASE WHEN account='C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS hy_target,
    SUM(CASE WHEN account='C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS hy_ya
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month BETWEEN '2025M07' AND '2025M12'
    AND quarter IN ('JAS','OND')
),

t_month_wide AS (
  SELECT
    'TOTAL' AS team,
    SUM(CASE WHEN month='2025M07' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END)/1000/1000/150.1 AS JULY_MM,
    SUM(CASE WHEN month='2025M07' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END) /
      NULLIF(SUM(CASE WHEN month='2025M07' AND account='C-NOS' THEN CAST(ya_actual AS DOUBLE) END),0) * 100 AS JULY_IYA,

    SUM(CASE WHEN month='2025M08' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END)/1000/1000/150.1 AS AUG_MM,
    SUM(CASE WHEN month='2025M08' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END) /
      NULLIF(SUM(CASE WHEN month='2025M08' AND account='C-NOS' THEN CAST(ya_actual AS DOUBLE) END),0) * 100 AS AUG_IYA,

    SUM(CASE WHEN month='2025M09' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END)/1000/1000/150.1 AS SEP_MM,
    SUM(CASE WHEN month='2025M09' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END) /
      NULLIF(SUM(CASE WHEN month='2025M09' AND account='C-NOS' THEN CAST(ya_actual AS DOUBLE) END),0) * 100 AS SEP_IYA,

    SUM(CASE WHEN month='2025M10' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END)/1000/1000/150.1 AS OCT_MM,
    SUM(CASE WHEN month='2025M10' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END) /
      NULLIF(SUM(CASE WHEN month='2025M10' AND account='C-NOS' THEN CAST(ya_actual AS DOUBLE) END),0) * 100 AS OCT_IYA,

    SUM(CASE WHEN month='2025M11' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END)/1000/1000/150.1 AS NOV_MM,
    SUM(CASE WHEN month='2025M11' AND account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) END) /
      NULLIF(SUM(CASE WHEN month='2025M11' AND account='C-NOS' THEN CAST(ya_actual AS DOUBLE) END),0) * 100 AS NOV_IYA,

    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') AND account='C-NOS'
             THEN CAST(forecast_act_est AS DOUBLE) END)/1000/1000/150.1 AS H1_to_date_MM,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') AND account='C-NOS'
             THEN CAST(forecast_act_est AS DOUBLE) END) /
      NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') AND account='C-NOS'
                      THEN CAST(ya_actual AS DOUBLE) END),0) * 100 AS H1_to_date_IYA
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month BETWEEN '2025M07' AND '2025M12'
    AND quarter IN ('JAS','OND')
),

total_metrics AS (
  SELECT
    m.team,
    MAX(CASE WHEN q.quarter = 'JAS' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS JAS_IYA,
    MAX(CASE WHEN q.quarter = 'OND' THEN q.q_target/NULLIF(q.q_ya,0)*100 END) AS OND_IYA,
    h.hy_target/1000/1000/150.1 AS `1H_MM`,
    h.hy_target/NULLIF(h.hy_ya,0)*100 AS `1H_IYA`,
    m.JULY_MM        AS JUL_MM,
    m.JULY_IYA       AS JUL_IYA,
    m.AUG_MM         AS AUG_MM,
    m.AUG_IYA        AS AUG_IYA,
    m.SEP_MM         AS SEP_MM,
    m.SEP_IYA        AS SEP_IYA,
    m.OCT_MM         AS OCT_MM,
    m.OCT_IYA        AS OCT_IYA,
    m.NOV_MM         AS NOV_MM,
    m.NOV_IYA        AS NOV_IYA,
    m.H1_to_date_MM  AS `1H_To_Date_MM`,
    m.H1_to_date_IYA AS `1H_To_Date_IYA`
  FROM t_month_wide m
  LEFT JOIN t_quarter q ON q.team = m.team
  LEFT JOIN t_half_year h ON h.team = m.team
  GROUP BY m.team, h.hy_target, h.hy_cnos, h.hy_ya,
           m.JULY_MM, m.JULY_IYA, m.AUG_MM, m.AUG_IYA,
           m.SEP_MM, m.SEP_IYA, m.OCT_MM, m.OCT_IYA, m.NOV_MM, m.NOV_IYA,
           m.H1_to_date_MM, m.H1_to_date_IYA
),

final_union_cnos AS (
  SELECT * FROM team_metrics
  UNION ALL
  SELECT * FROM total_metrics
),

/* ------------ GIV path (1H-to-date IYA) with same buckets & no raw 'Division 2' in detail; labels Div 1/Div 2 ------------ */
giv_base AS (
  SELECT
    org2,
    org4,
    month,
    SUM(CAST(forecast_act_est AS DOUBLE)) AS giv,
    SUM(CAST(ya_actual        AS DOUBLE)) AS giv_ya
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month BETWEEN '2025M07' AND '2025M12'
    AND account = 'GIV'
  GROUP BY org2, org4, month
),

giv_detail AS (
  SELECT
    CASE
      WHEN org2 = 'Division 2' AND org4 IN ('DP 1','DP 2','DP 3','DP 4','DP 5') THEN '   DP'
      WHEN org2 = 'Division 2' AND org4 IN ('MHC 1','MHC 2','MHC 3','MHC 4','MHC 5') THEN '   MHC'
      WHEN org2 = 'Division 2' AND org4 IN ('HSM 1','HSM 2','HSM 3')              THEN '   HSM'
      WHEN org2 = 'Division 2' AND org4 IN ('WS 1','WS 2','WS 3')                 THEN '   WS'
      WHEN org2 = 'Division 1'                                                  THEN 'Div 1'
      WHEN org2 <> 'Division 2' AND org2 <> 'Division 1'                         THEN org2
      ELSE NULL
    END AS team,
    month,
    SUM(giv)    AS giv,
    SUM(giv_ya) AS giv_ya
  FROM giv_base
  GROUP BY
    CASE
      WHEN org2 = 'Division 2' AND org4 IN ('DP 1','DP 2','DP 3','DP 4','DP 5') THEN '   DP'
      WHEN org2 = 'Division 2' AND org4 IN ('MHC 1','MHC 2','MHC 3','MHC 4','MHC 5') THEN '   MHC'
      WHEN org2 = 'Division 2' AND org4 IN ('HSM 1','HSM 2','HSM 3')              THEN '   HSM'
      WHEN org2 = 'Division 2' AND org4 IN ('WS 1','WS 2','WS 3')                 THEN '   WS'
      WHEN org2 = 'Division 1'                                                  THEN 'Div 1'
      WHEN org2 <> 'Division 2' AND org2 <> 'Division 1'                         THEN org2
      ELSE NULL
    END,
    month
),

giv_detail_clean AS (
  SELECT * FROM giv_detail WHERE team IS NOT NULL
),

giv_div2_total AS (
  SELECT
    'Div 2' AS team,  -- label changed
    month,
    SUM(giv)    AS giv,
    SUM(giv_ya) AS giv_ya
  FROM giv_base
  WHERE org2 = 'Division 2'  -- keep raw filter
  GROUP BY month
),

giv_total AS (
  SELECT
    'TOTAL' AS team,
    month,
    SUM(giv)    AS giv,
    SUM(giv_ya) AS giv_ya
  FROM giv_base
  GROUP BY month
),

giv_all AS (
  SELECT * FROM giv_detail_clean
  UNION ALL
  SELECT * FROM giv_div2_total
  UNION ALL
  SELECT * FROM giv_total
),

giv_final AS (
  SELECT
    team,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv END)
    / NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN giv_ya END), 0) * 100
      AS GIV_1H_to_date_IYA
  FROM giv_all
  GROUP BY team
)

/* ------------ Merge C‑NOS metrics with GIV IYA ------------ */
SELECT
  c.team AS `By Team`,
  ROUND(c.JAS_IYA, 0)        AS JAS_IYA,
  ROUND(c.OND_IYA, 0)        AS OND_IYA,
  ROUND(c.`1H_MM`, 0)        AS `1H_MM`,
  ROUND(c.`1H_IYA`, 0)        AS 1H_IYA,
  ROUND(c.JUL_MM, 0)         AS JUL_MM,
  ROUND(c.JUL_IYA, 0)        AS JUL_IYA,
  ROUND(c.AUG_MM, 0)         AS AUG_MM,
  ROUND(c.AUG_IYA, 0)        AS AUG_IYA,
  ROUND(c.SEP_MM, 0)         AS SEP_MM,
  ROUND(c.SEP_IYA, 0)        AS SEP_IYA,
  ROUND(c.OCT_MM, 0)         AS OCT_MM,
  ROUND(c.OCT_IYA, 0)        AS OCT_IYA,
  ROUND(c.NOV_MM, 0)         AS NOV_MM,
  ROUND(c.NOV_IYA, 0)        AS NOV_IYA,
  ROUND(c.`1H_To_Date_MM`, 0)         AS `1H_To_Date_MM`,
  ROUND(c.`1H_To_Date_IYA`, 0)        AS `1H_To_Date_IYA`,
  ROUND(c.`1H_To_Date_IYA` - c.`1H_IYA`, 0) AS `vs_Target`,
  ROUND(g.GIV_1H_to_date_IYA, 0)            AS `GIV_1H_to_date_IYA`,
  ROUND(c.`1H_To_Date_IYA` - g.GIV_1H_to_date_IYA, 0) AS `Gap_VS._C-NOS`
FROM final_union_cnos c
LEFT JOIN giv_final g
  ON g.team = c.team
ORDER BY
  CASE c.team
    WHEN 'TOTAL' THEN 0
    WHEN 'Div 1' THEN 1      -- updated label
    WHEN 'Div 2' THEN 2      -- updated label
    WHEN '   DP'    THEN 3
    WHEN '   MHC'   THEN 4
    WHEN '   HSM'   THEN 5
    WHEN '   WS'    THEN 6
    WHEN 'D-Com' THEN 7
    WHEN 'Electro' THEN 8
    ELSE 99
  END, c.team ASC;

In [0]:
%sql

-- By Customer --

WITH base AS (
  SELECT customer, org2, month, quarter, account, forecast_act_est, target, ya_actual
  FROM hive_metastore.japan_gold.cnos_vw
  WHERE month BETWEEN '2025M07' AND '2025M12'
    AND quarter IN ('JAS','OND')
    AND channel != 'Baby Specialty'
),

-- Filtered customers (Query 1 restrictions)
div1_cust AS (
  SELECT * FROM base
  WHERE org2 = 'Division 1'
    AND customer IN (
      'FN-WELCIA YAKKYOKU','FN-CAINZ(C003)','FN-MATSUKIYO_COCOKARA (B008)','FN-SUGI_YAKKYOKU(I015)',
      'FN-AEON_GROUP(JUSCOJ4)(C005)','FN-COSMOS_YAKUHIN(W030)','FN-COSTCO(A062)','FN-TSURUHA_HOKK+TOHO(E030)',
      'FN-DONKIHOTE(A094)','FN-SUN_DRUG(D005)','FN-TGN','FN-KYORINDO(H011)','FN-LADY_YAKKYOKU(K067)',
      'FN-KUSURINO FUKUTARO (G043)','FN-ELEVEN(L002)','FN-UNY(B009)','FN-SUNDAY(E015)'
    )
),
dcom_cust AS (
  SELECT * FROM base
  WHERE org2 = 'D-Com'
    AND customer IN ('FN-AMAZON JAPAN','FN-RAKUTEN')
),

-- Customer-level monthly aggregation (C-NOS)
cust_month_c AS (
  SELECT
    customer,
    month,
    SUM(CASE WHEN account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS cnos_ya
  FROM (
    SELECT customer, month, account, forecast_act_est, target, ya_actual FROM div1_cust
    UNION ALL
    SELECT customer, month, account, forecast_act_est, target, ya_actual FROM dcom_cust
  )
  GROUP BY customer, month
),

-- Org-level monthly rollup (built ONLY from filtered customers)
org_month AS (
  SELECT
    'Division 1' AS customer,
    month,
    SUM(CASE WHEN account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS cnos_ya
  FROM div1_cust
  GROUP BY month

  UNION ALL

  SELECT
    'D-Com' AS customer,
    month,
    SUM(CASE WHEN account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS cnos_ya
  FROM dcom_cust
  GROUP BY month
),

-- Customer-level quarterly aggregation (C-NOS)
cust_quarter_c AS (
  SELECT
    customer,
    quarter,
    SUM(CASE WHEN account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS cnos_ya
  FROM (
    SELECT customer, quarter, account, forecast_act_est, target, ya_actual FROM div1_cust
    UNION ALL
    SELECT customer, quarter, account, forecast_act_est, target, ya_actual FROM dcom_cust
  )
  GROUP BY customer, quarter
),

-- Org-level quarterly rollup (built ONLY from filtered customers)
org_quarter AS (
  SELECT
    'Division 1' AS customer,
    quarter,
    SUM(CASE WHEN account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS cnos_ya
  FROM div1_cust
  GROUP BY quarter

  UNION ALL

  SELECT
    'D-Com' AS customer,
    quarter,
    SUM(CASE WHEN account='C-NOS' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS cnos,
    SUM(CASE WHEN account='C-NOS' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS cnos_target,
    SUM(CASE WHEN account='C-NOS' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS cnos_ya
  FROM dcom_cust
  GROUP BY quarter
),

-- Unified monthly & quarterly datasets (customer + org)
unified_month AS (
  SELECT * FROM cust_month_c
  UNION ALL
  SELECT * FROM org_month
),
unified_quarter AS (
  SELECT * FROM cust_quarter_c
  UNION ALL
  SELECT * FROM org_quarter
),

-- Quarterly pivot & 1H metrics (C-NOS)
q_pivot AS (
  SELECT
    customer,
    MAX(CASE WHEN quarter = 'JAS' THEN cnos_target / NULLIF(cnos_ya,0) * 100 END) AS JAS_IYA,
    MAX(CASE WHEN quarter = 'OND' THEN cnos_target / NULLIF(cnos_ya,0) * 100 END) AS OND_IYA,
    SUM(cnos_target) / 1000 / 1000 / 150.1 AS `1H_MM`,
    SUM(cnos_target) / NULLIF(SUM(cnos_ya),0) * 100 AS `1H_IYA`
  FROM unified_quarter
  GROUP BY customer
),

-- Monthly pivot & 1H-to-date (C-NOS)
m_pivot AS (
  SELECT
    customer,
    MAX(CASE WHEN month = '2025M07' THEN cnos END)/1000/1000/150.1 AS JULY_MM,
    MAX(CASE WHEN month = '2025M07' THEN cnos/cnos_ya*100 END)     AS JULY_IYA,
    MAX(CASE WHEN month = '2025M08' THEN cnos END)/1000/1000/150.1 AS AUG_MM,
    MAX(CASE WHEN month = '2025M08' THEN cnos/cnos_ya*100 END)     AS AUG_IYA,
    MAX(CASE WHEN month = '2025M09' THEN cnos END)/1000/1000/150.1 AS SEP_MM,
    MAX(CASE WHEN month = '2025M09' THEN cnos/cnos_ya*100 END)     AS SEP_IYA,
    MAX(CASE WHEN month = '2025M10' THEN cnos END)/1000/1000/150.1 AS OCT_MM,
    MAX(CASE WHEN month = '2025M10' THEN cnos/cnos_ya*100 END)     AS OCT_IYA,
    MAX(CASE WHEN month = '2025M11' THEN cnos END)/1000/1000/150.1 AS NOV_MM,
    MAX(CASE WHEN month = '2025M11' THEN cnos/cnos_ya*100 END)     AS NOV_IYA,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END)/1000/1000/150.1 AS `1H_To_Date_MM`,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos END)/
          NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN cnos_ya END),0)*100 AS `1H_to_date_IYA`
  FROM unified_month
  GROUP BY customer
),

-- GIV monthly (customer + org)
giv_month_c AS (
  SELECT
    customer,
    month,
    SUM(CASE WHEN account='GIV' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS GIV,
    SUM(CASE WHEN account='GIV' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS GIV_target,
    SUM(CASE WHEN account='GIV' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS GIV_ya
  FROM (
    SELECT customer, month, account, forecast_act_est, target, ya_actual FROM div1_cust
    UNION ALL
    SELECT customer, month, account, forecast_act_est, target, ya_actual FROM dcom_cust
  )
  GROUP BY customer, month
),
giv_month_org AS (
  SELECT
    'Division 1' AS customer,
    month,
    SUM(CASE WHEN account='GIV' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS GIV,
    SUM(CASE WHEN account='GIV' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS GIV_target,
    SUM(CASE WHEN account='GIV' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS GIV_ya
  FROM div1_cust
  GROUP BY month

  UNION ALL

  SELECT
    'D-Com' AS customer,
    month,
    SUM(CASE WHEN account='GIV' THEN CAST(forecast_act_est AS DOUBLE) ELSE 0 END) AS GIV,
    SUM(CASE WHEN account='GIV' THEN CAST(target           AS DOUBLE) ELSE 0 END) AS GIV_target,
    SUM(CASE WHEN account='GIV' THEN CAST(ya_actual        AS DOUBLE) ELSE 0 END) AS GIV_ya
  FROM dcom_cust
  GROUP BY month
),
giv_unified AS (
  SELECT * FROM giv_month_c
  UNION ALL
  SELECT * FROM giv_month_org
),
giv_pivot AS (
  SELECT
    customer,
    SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN GIV END) /
          NULLIF(SUM(CASE WHEN month IN ('2025M07','2025M08','2025M09','2025M10','2025M11') THEN GIV_ya END),0) * 100 AS GIV_1H_to_date_IYA
  FROM giv_unified
  GROUP BY customer
),

final AS (
  SELECT
    COALESCE(q.customer, m.customer) AS customer,
    q.JAS_IYA, q.OND_IYA, q.`1H_MM`, q.`1H_IYA`,
    m.JULY_MM, m.JULY_IYA, m.AUG_MM, m.AUG_IYA,
    m.SEP_MM, m.SEP_IYA, m.OCT_MM, m.OCT_IYA, m.NOV_MM, m.NOV_IYA,
    m.`1H_To_Date_MM`, m.`1H_to_date_IYA`,
    (q.`1H_IYA` - m.`1H_to_date_IYA`) AS `1H_To_Date_VS_1H_Target`
  FROM q_pivot q
  FULL OUTER JOIN m_pivot m
    ON q.customer = m.customer
),

-- ===== Label & Order as per reference =====
labeled AS (
  SELECT
    -- Display names exactly as per reference (group headers + short customer labels)
    CASE
      WHEN f.customer = 'Division 1'             THEN 'Div 1 (Ex. Baby Specialty)'
      WHEN f.customer = 'D-Com'                  THEN 'D Com'
      WHEN f.customer = 'FN-WELCIA YAKKYOKU'     THEN '   WELCIA'
      WHEN f.customer = 'FN-CAINZ(C003)'         THEN '   CAINZ'
      WHEN f.customer = 'FN-MATSUKIYO_COCOKARA (B008)' THEN '   MCC'
      WHEN f.customer = 'FN-SUGI_YAKKYOKU(I015)' THEN '   SUGI'
      WHEN f.customer = 'FN-AEON_GROUP(JUSCOJ4)(C005)' THEN '   AEON'
      WHEN f.customer = 'FN-COSMOS_YAKUHIN(W030)' THEN '   COSMOS'
      WHEN f.customer = 'FN-COSTCO(A062)'        THEN '   COSTCO'
      WHEN f.customer = 'FN-TSURUHA_HOKK+TOHO(E030)' THEN '   TSURUHA'
      WHEN f.customer = 'FN-DONKIHOTE(A094)'     THEN '   DONKI'
      WHEN f.customer = 'FN-SUN_DRUG(D005)'      THEN '   SUNDRUG'
      WHEN f.customer = 'FN-TGN'                 THEN '   TGN'
      WHEN f.customer = 'FN-KYORINDO(H011)'      THEN '   KYORINDO'
      WHEN f.customer = 'FN-LADY_YAKKYOKU(K067)' THEN '   LADY'
      WHEN f.customer = 'FN-KUSURINO FUKUTARO (G043)' THEN '   FUKUTARO'
      WHEN f.customer = 'FN-ELEVEN(L002)'        THEN '   ELEVEN'
      WHEN f.customer = 'FN-UNY(B009)'           THEN '   UNY'
      WHEN f.customer = 'FN-SUNDAY(E015)'        THEN '   SUNDAY'
      WHEN f.customer = 'FN-AMAZON JAPAN'        THEN '   Amazon'
      WHEN f.customer = 'FN-RAKUTEN'             THEN '   Rakuten'
      ELSE f.customer
    END AS `By Customer`,

    -- Metrics (rounded, unchanged logic)
    ROUND(f.JAS_IYA, 0)        AS JAS_IYA,
    ROUND(f.OND_IYA, 0)        AS OND_IYA,
    ROUND(f.`1H_MM`, 0)        AS `1H_MM`,
    ROUND(f.`1H_IYA`, 0)       AS 1H_IYA,
    ROUND(f.JULY_MM, 0)        AS JUL_MM,
    ROUND(f.JULY_IYA, 0)       AS JUL_IYA,
    ROUND(f.AUG_MM, 0)         AS AUG_MM,
    ROUND(f.AUG_IYA, 0)        AS AUG_IYA,
    ROUND(f.SEP_MM, 0)         AS SEP_MM,
    ROUND(f.SEP_IYA, 0)        AS SEP_IYA,
    ROUND(f.OCT_MM, 0)         AS OCT_MM,
    ROUND(f.OCT_IYA, 0)        AS OCT_IYA,
    ROUND(f.NOV_MM, 0)         AS NOV_MM,
    ROUND(f.NOV_IYA, 0)        AS NOV_IYA,
    ROUND(f.`1H_To_Date_MM`, 0)   AS 1H_To_Date_MM,
    ROUND(f.`1H_to_date_IYA`, 0)  AS 1H_To_Date_IYA,

    ROUND(f.`1H_to_date_IYA` - f.`1H_IYA`, 0) AS `vs_Target`,
    ROUND(g.GIV_1H_to_date_IYA, 0)            AS `GIV_1H_to_date_IYA`,
    ROUND(f.`1H_to_date_IYA` - g.GIV_1H_to_date_IYA, 0) AS `Gap_VS._C-NOS`
  FROM final f
  LEFT JOIN giv_pivot g
    ON f.customer = g.customer
)

SELECT *
FROM labeled
ORDER BY
  CASE `By Customer`
    WHEN 'Div 1 (Ex. Baby Specialty)' THEN 0
    WHEN '   WELCIA'   THEN 1
    WHEN '   CAINZ'    THEN 2
    WHEN '   MCC'      THEN 3
    WHEN '   SUGI'     THEN 4
    WHEN '   AEON'     THEN 5
    WHEN '   COSMOS'   THEN 6
    WHEN '   COSTCO'   THEN 7
    WHEN '   TSURUHA'  THEN 8
    WHEN '   DONKI'    THEN 9
    WHEN '   SUNDRUG'  THEN 10
    WHEN '   TGN'      THEN 11
    WHEN '   KYORINDO' THEN 12
    WHEN '   LADY'     THEN 13
    WHEN '   FUKUTARO' THEN 14
    WHEN '   ELEVEN'   THEN 15
    WHEN '   UNY'      THEN 16
    WHEN '   SUNDAY'   THEN 17
    WHEN 'D Com'    THEN 18
    WHEN '   Amazon'   THEN 19
    WHEN '   Rakuten'  THEN 20
    ELSE 99
  END,
  `By Customer`;